### Descarga Realtime Alpha Vantage
###### Descarga cada 10 minutos

In [1]:
import pandas as pd
import numpy as np
import time
#import matplotlib.pyplot as plt
import os
from datetime import date, datetime,timedelta
#from dateutil.relativedelta import relativedelta

In [2]:
# Verificar si el archivo existe
ruta_archivo='data/dataxd.txt'
if os.path.exists(ruta_archivo):
    # Cargar el archivo
    df1 = pd.read_csv(ruta_archivo, sep='\t')
    print("Archivo cargado correctamente.")
else:
    # Crear un DataFrame vacío
    df1 = pd.DataFrame()
    print("Archivo no existe. Se creó un DataFrame vacío.")

Archivo cargado correctamente.


In [3]:
ruta_archivo='data/dataxh.txt'
if os.path.exists(ruta_archivo):
    # Cargar el archivo
    df2 = pd.read_csv(ruta_archivo, sep='\t')
    print("Archivo cargado correctamente.")
else:
    # Crear un DataFrame vacío
    df2 = pd.DataFrame()
    print("Archivo no existe. Se creó un DataFrame vacío.")

Archivo cargado correctamente.


In [4]:
df1['datetime'] = pd.to_datetime(df1['datetime'])
df2['datetime'] = pd.to_datetime(df2['datetime'])

In [5]:
max_day1 = (df1['datetime'].max()).date()
max_day2 = (df2['datetime'].max()).date()

In [6]:
max_day2 = max_day2 - timedelta(days=5) #para Realtime restar 5 dias

In [7]:
#today = date.today()
max_day_str2 = str(max_day2)
max_day_str1 = str(max_day1)
tickers = [
    'SPY',
    'META',
    'AAPL',
    'AMZN',
    'NFLX',
    'MRNA',
    'TSLA',
    'TNA',
    'GLD',
    'SLV',
    'USO',
    'BAC',
    'CVX',
    'XOM',
    'QQQ',
    'MSFT',
    'NVDA',
    'WMT',
    'BA',
    'DIS',
    'CAT',
    'IBM',
    'WFC',
    'PLTR',
    'AMD',
    'AVGO',
    'HOOD',
    'CRWV',
    'MSTR',
    'UNH',
    'GOOG',
    'APP',
    'UBER'
]

In [8]:
#INFORMACION DIAS
df_new1 = pd.DataFrame()
df_new1 = df1.copy()
for ticker in tickers:
    url1='https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol='+str(ticker)+'&entitlement=delayed&apikey=HR9C4MJVM7O4UUCM&datatype=csv'
    company = pd.read_csv(url1, sep=',')
    company=company.rename(columns={'timestamp':'datetime','open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close', 'volume': 'Volume'})
    company = company.drop(['adjusted_close','dividend_amount','split_coefficient'], axis=1)
    company['companyName']=ticker
    company = company[['Close', 'High', 'Low', 'Open', 'Volume', 'datetime', 'companyName']] #ordenar columnas
    company['datetime'] = pd.to_datetime(company['datetime'])
    company_lastday = company[company['datetime']>max_day_str1]
    df_new1 = pd.concat([df_new1, company_lastday],ignore_index=True)

In [9]:
#Funcion limpiar OUTLIERS
def df_limpiar(df_eval):
    #dataframe para obtener medias sin incluir fecha actual
    df_eval2 = df_eval[df_eval['datetime']<max_day_str2].tail(100)
    
    meandif1 = df_eval2['dif1'].mean()
    stddif1 = df_eval2['dif1'].std()
    topdif1 = meandif1 + stddif1 * 1.96

    meandif2 = df_eval2['dif2'].mean()
    stddif2 = df_eval2['dif2'].std()
    topdif2 = meandif2 + stddif2 * 1.96
    
    #botdif = meandif - stddif * 3
    copydf = df_eval.copy()
    
    #df_h.apply(lambda row: row['High'] - row['Close'] if row['Close'] > row['Open'] else row['High'] - row['Open'], axis=1)
    copydf['ind1'] = copydf.apply(lambda row: 1 if row['dif1'] > topdif1 else 0, axis=1)
    copydf['ind2'] = copydf.apply(lambda row: 1 if row['dif2'] > topdif2 else 0, axis=1)

    #hoy = datetime.now()    
    #mes_actual = hoy.replace(day=1)
    #mes_fin = mes_actual - relativedelta(months=3)
    #hace_12_meses = mes_actual - relativedelta(months=12)
    #cuartil 3
    q3_dif1 = copydf[copydf['ind1']!=1]['dif1'].quantile(0.75)
    q3_dif2 = copydf[copydf['ind2']!=1]['dif2'].quantile(0.75)
    
    #meanfin1 = copydf[(copydf['ind1']==1) & (copydf['datetime'] >= hace_12_meses) & (copydf['datetime'] < mes_fin)]['dif1'].mean()
    #meanfin2 = copydf[(copydf['ind2']==1) & (copydf['datetime'] >= hace_12_meses) & (copydf['datetime'] < mes_fin)]['dif2'].mean()

    print("q3_dif1:",q3_dif1)
    print("q3_dif2:",q3_dif2)

    copydf['High2']=copydf.apply(
        lambda row: row['Close'] + q3_dif1 if (row['Close'] > row['Open']) & (row['ind1']==1) & (row['eval']==0)  else 
        row['Open'] + q3_dif1 if (row['Close'] <= row['Open']) & (row['ind1']==1) & (row['eval']==0) else
        row['High'], axis=1)
    copydf['Low2']=copydf.apply(
        lambda row: row['Open'] - q3_dif2 if (row['Close'] > row['Open']) & (row['ind2']==1) & (row['eval']==0) else 
        row['Close'] - q3_dif2 if (row['Close'] <= row['Open']) & (row['ind2']==1) & (row['eval']==0) else
        row['Low'], axis=1)
   
    #copydf = copydf.drop(copydf[copydf['dif1'] > topdif1].index)
    #copydf = copydf.drop(copydf[copydf['dif2'] > topdif2].index)
    return copydf

In [10]:
df_new2 = pd.DataFrame()
for ticker in tickers:
    #Informacion anterior por ticker
    df_ticker = df2[(df2['datetime'] <  max_day_str2) & (df2['companyName']==ticker)]
    df_ticker = df_ticker.reset_index(drop=True)
    df_ticker['datetime'] = pd.to_datetime(df_ticker['datetime'])
    #crear columnas de diferencia
    df_ticker['dif1'] = df_ticker.apply(lambda row: row['High'] - row['Close'] if row['Close'] > row['Open'] else row['High'] - row['Open'], axis=1)
    df_ticker['dif2'] = df_ticker.apply(lambda row: row['Open'] - row['Low'] if row['Close'] > row['Open'] else row['Close'] - row['Low'], axis=1)
    df_ticker['eval'] = 1 #no se limpiara
    #Aplha vantage descarga intraday
    url='https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol=' + str(ticker)+ '&interval=60min&entitlement=delayed&apikey=HR9C4MJVM7O4UUCM&datatype=csv'
    company = pd.read_csv(url, sep=',')         
    company = company[['close','high','low','open','volume','timestamp']]
    company=company.rename(columns={"open": "Open","high": "High","low": "Low","close": "Close","volume": "Volume","timestamp": "datetime"})        
    company['companyName']=ticker
    company['datetime'] = pd.to_datetime(company['datetime'])
    company['datetime'] = company['datetime'].dt.tz_localize('UTC')
    #company['datetime'] = company['datetime'].dt.tz_localize('US/Eastern')
    company_lastday = company[company['datetime']>=max_day_str2]
    company_lastday = company_lastday.sort_values(by=['datetime'], ascending=[True])
    
    #crear columnas diferencia de las colas (se evaluara el ultimo dia)
    company_lastday['dif1'] = company_lastday.apply(lambda row: row['High'] - row['Close'] if row['Close'] > row['Open'] else row['High'] - row['Open'], axis=1)
    company_lastday['dif2'] = company_lastday.apply(lambda row: row['Open'] - row['Low'] if row['Close'] > row['Open'] else row['Close'] - row['Low'], axis=1)
    company_lastday['eval'] = 0 # se limpiara    
    df_ticker = pd.concat([df_ticker, company_lastday],ignore_index=True)    
    company_final = df_limpiar(df_ticker)    
    df_new2 = pd.concat([df_new2, company_final],ignore_index=True)

q3_dif1: 0.5
q3_dif2: 0.6049880981445312


q3_dif1: 1.3271331787109375
q3_dif2: 1.3500289916992188


q3_dif1: 0.4100036621093466
q3_dif2: 0.3849945068359375


q3_dif1: 0.4199981689453125
q3_dif2: 0.43999099731447444


q3_dif1: 1.80999755859375
q3_dif2: 1.8300170898438353


...

q3_dif1: 1.286895751953125
q3_dif2: 1.199542999267571


q3_dif1: 0.885009765625
q3_dif2: 0.895172119140625


q3_dif1: 0.3300018310546875
q3_dif2: 0.3699951171875


q3_dif1: 0.6860027313232422
q3_dif2: 0.7400093078613281


q3_dif1: 0.1999969482421875
q3_dif2: 0.20000457763671875


In [11]:
df_new2['High'] = np.where(df_new2['eval'] == 0, df_new2['High2'], df_new2['High'])
df_new2['Low'] = np.where(df_new2['eval'] == 0, df_new2['Low2'], df_new2['Low'])

In [12]:
df_new2 = df_new2.drop(columns=['dif1', 'dif2', 'eval', 'ind1', 'ind2', 'High2','Low2' ])

In [13]:
#descarga de informacion, estrategia: RUPTURA DEL CANAL BAJISTA
path='data/dataxd.txt'
folder=os.path.dirname(path)

if not os.path.exists(folder):
    os.makedirs(folder)

if os.path.exists(path):
    # delete the file
    os.remove(path)
else:
    # if the file does not exist.
    print("File does not exists. File needs to be created.")

#export DataFrame to text file
with open(path, 'a') as f:
    #df_string = appl_hor3.to_string(header=True, index=False, sep ='\t')
    df_new1.to_csv(path, header=True, index=None, sep='\t', mode='w')
    #f.write(df_string)

In [14]:
#descarga de informacion
path='data/dataxh.txt'
# check whether the file exists
if os.path.exists(path):
    # delete the file
    os.remove(path)
else:
    # if the file does not exist.
    print("File does not exists. File needs to be created.")

#export DataFrame to text file
with open(path, 'a') as f:
    #df_string = appl_hor3.to_string(header=True, index=False, sep ='\t')
    df_new2.to_csv(path, header=True, index=None, sep='\t', mode='w')
    #f.write(df_string)